In [1]:
import itertools
from parlay_system import *
pd.set_option('display.max_rows', 500)


In [2]:
# Dec 15 games

# line 7
ut_rio = MoneyLine(event="ut_rio", bet_amount=100, odds=300)
omaha = MoneyLine(event="omaha", bet_amount=100, odds=-385)

# line 6.5
oklast = MoneyLine(event="oklast", bet_amount=100, odds=235)
houston = MoneyLine(event="houston", bet_amount=100, odds=-295)

# line 5
ga_south = MoneyLine(event="ga_south", bet_amount=100, odds=180)
bradley = MoneyLine(event="bradley", bet_amount=100, odds=-225)

# line 7
samford = MoneyLine(event="samford", bet_amount=100, odds=195)
hawaii = MoneyLine(event="hawaii", bet_amount=100, odds=-245)

# line 5.5
sc = MoneyLine(event="sc", bet_amount=100, odds=215)
clemson = MoneyLine(event="clemson", bet_amount=100, odds=-275)

#line 3.5
quinnipiac = MoneyLine(event="quinnipiac", bet_amount=100, odds=-175)
maine = MoneyLine(event="maine", bet_amount=100, odds=143)

# line 6
coppinst = MoneyLine(event="coppinst", bet_amount=100, odds=900)
umbc = MoneyLine(event="umba", bet_amount=100, odds=-1667)

# line 5.5
illst = MoneyLine(event="illst", bet_amount=100, odds=200)
nkentucky = MoneyLine(event="nkentucky", bet_amount=100, odds=-245)

#line 3.5
tennessee = MoneyLine(event="tennessee", bet_amount=100, odds=143)
fordham = MoneyLine(event="fordham", bet_amount=100, odds=-175)

# line 2.5
ncstate = MoneyLine(event="ncstate", bet_amount=100, odds=-148)
uncgreen = MoneyLine(event="uncgreen", bet_amount=100, odds=120)

binaries = [
#     [ut_rio, omaha],
#     [oklast, houston],
    [ga_south, bradley],
    [samford, hawaii],
    [sc, clemson],
    [quinnipiac, maine],
#     [coppinst, umbc],
#     [illst, nkentucky],
    [tennessee, fordham],
    [ncstate, uncgreen]
]

results = [
#     [0, 1], #[ut_rio, omaha],
#     [0, 1], #[oklast, houston],
    [1, 0], #[ga_south, bradley],
    [0, 1], #[samford, hawaii],
    [0, 1], #[sc, clemson],
    [0, 1], #[quinnipiac, maine],
#     [0, 1], #[coppinst, umbc],
#     [0, 1], #[illst, nkentucky],
    [1, 0], #[tennessee, fordham],
    [1, 0], #[ncstate, uncgreen]
]


def create_dicts(in_binaries, in_results):
    ml_lookup = {}
    ml_win_lookup = {}
    id = 0
    for bi in range(len(in_binaries)):
        binary = in_binaries[bi]
        for ml_i in range(len(binary)):
            ml = binary[ml_i]
            ml.set_index(id) 
            is_winner = in_results[bi][ml_i]
            ml_lookup[id] = ml.event
            ml_win_lookup[id] = is_winner
            id += 1
    
    return ml_lookup, ml_win_lookup

ml_lookup, ml_outcome_lookup = create_dicts(binaries, results)



In [3]:
#ml_lookup

In [4]:
#ml_outcome_lookup

In [5]:
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

TOURNEY_SIZE = len(binaries)
SELECT_NUM = 3
s = set()
n = SELECT_NUM

for i in range(TOURNEY_SIZE):
    s.add(i)

all_subsets = findsubsets(s, n)

In [6]:
len(all_subsets)

20

In [7]:
csv = []
df = pd.DataFrame()

for i in range(len(all_subsets)):
    current_subset = all_subsets[i]
    parlay_binaries = []
    for i in range(len(current_subset)):
        index = current_subset[i]
        parlay_binaries.append(binaries[index])

    ps = ParlaySystem(binaries=parlay_binaries,
                    target_profit=0.05,
                    bounds=(0.01, 30),
                    binary_index_arr=list(current_subset),
                    binary_results_arr=results,
                    index_to_ml=ml_lookup,
                    index_to_outcome=ml_outcome_lookup
                    )
    csv_res, raw_df = ps.slsqp_solver()
    df = pd.concat([df, raw_df])
    csv.append(csv_res)

     fun: -0.037533229641152854
     jac: array([-2.377375  , -0.53295455, -0.6775    ,  0.20292208, -0.8028125 ,
        0.14867424,  0.07410714,  0.52829314])
 message: 'Positive directional derivative for linesearch'
    nfev: 195
     nit: 16
    njev: 12
  status: 8
 success: False
       x: array([0.01      , 0.01473272, 0.01336103, 0.01783986, 0.01236312,
       0.01846379, 0.0191835 , 0.01      ])
slsqp_solver: 
                       event    index[]     result  event_status   bet  \
3   ga_south_hawaii_clemson_  [0, 3, 5]  [1, 1, 1]          True  0.02   
0       ga_south_samford_sc_  [0, 2, 4]  [1, 0, 0]         False  0.01   
1  ga_south_samford_clemson_  [0, 2, 5]  [1, 0, 1]         False  0.01   
2        ga_south_hawaii_sc_  [0, 3, 4]  [1, 1, 0]         False  0.01   
4        bradley_samford_sc_  [1, 2, 4]  [0, 0, 0]         False  0.01   
6         bradley_hawaii_sc_  [1, 3, 4]  [0, 1, 0]         False  0.02   
5   bradley_samford_clemson_  [1, 2, 5]  [0, 0, 1]        

slsqp_solver: 
                         event    index[]     result  event_status   bet  \
2  ga_south_clemson_tennessee_  [0, 5, 8]  [1, 1, 1]          True  0.02   
0       ga_south_sc_tennessee_  [0, 4, 8]  [1, 0, 1]         False  0.01   
1         ga_south_sc_fordham_  [0, 4, 9]  [1, 0, 0]         False  0.01   
4        bradley_sc_tennessee_  [1, 4, 8]  [0, 0, 1]         False  0.02   
5          bradley_sc_fordham_  [1, 4, 9]  [0, 0, 0]         False  0.02   
3    ga_south_clemson_fordham_  [0, 5, 9]  [1, 1, 0]         False  0.02   
6   bradley_clemson_tennessee_  [1, 5, 8]  [0, 1, 1]         False  0.02   
7     bradley_clemson_fordham_  [1, 5, 9]  [0, 1, 0]         False  0.01   

        mult  payout  profit  
2   9.278182  0.1730    0.05  
0  21.432600  0.2143    0.09  
1  13.860000  0.1730    0.05  
4  11.056500  0.1730    0.05  
5   7.150000  0.1472    0.02  
3   6.000000  0.1142   -0.01  
6   4.786364  0.0794   -0.04  
7   3.095238  0.0310   -0.09  
total_bet   :  0.13
m

     fun: -0.032702056309730415
     jac: array([-0.9503125 , -1.94759688,  0.08482143, -0.34690341,  0.00369898,
       -0.47234821,  0.49781341,  0.29173237])
 message: 'Positive directional derivative for linesearch'
    nfev: 558
     nit: 37
    njev: 33
  status: 8
 success: False
       x: array([0.01171321, 0.01      , 0.01940337, 0.01749751, 0.02020295,
       0.01586841, 0.01      , 0.01635677])
slsqp_solver: 
                         event    index[]     result  event_status   bet  \
7        hawaii_clemson_maine_  [3, 5, 7]  [1, 1, 1]          True  0.02   
1            samford_sc_maine_  [2, 4, 7]  [0, 0, 1]         False  0.01   
0       samford_sc_quinnipiac_  [2, 4, 6]  [0, 0, 0]         False  0.01   
3       samford_clemson_maine_  [2, 5, 7]  [0, 1, 1]         False  0.02   
5             hawaii_sc_maine_  [3, 4, 7]  [1, 0, 1]         False  0.02   
4        hawaii_sc_quinnipiac_  [3, 4, 6]  [1, 0, 0]         False  0.02   
2  samford_clemson_quinnipiac_  [2, 5, 6]  [

     fun: -0.022762901257000562
     jac: array([-0.16182432, -0.48625   , -0.72830743, -1.2299875 ,  0.4261583 ,
        0.28571429,  0.18092752, -0.03625   ])
 message: 'Positive directional derivative for linesearch'
    nfev: 193
     nit: 17
    njev: 13
  status: 8
 success: False
       x: array([0.0209474 , 0.01595502, 0.01354623, 0.01031777, 0.01      ,
       0.01498383, 0.0173885 , 0.02061144])
slsqp_solver: 
                          event     index[]     result  event_status   bet  \
6        clemson_maine_ncstate_  [5, 7, 10]  [1, 1, 1]          True  0.02   
0        sc_quinnipiac_ncstate_  [4, 6, 10]  [0, 0, 1]         False  0.02   
1       sc_quinnipiac_uncgreen_  [4, 6, 11]  [0, 0, 0]         False  0.02   
2             sc_maine_ncstate_  [4, 7, 10]  [0, 1, 1]         False  0.01   
3            sc_maine_uncgreen_  [4, 7, 11]  [0, 1, 0]         False  0.01   
7       clemson_maine_uncgreen_  [5, 7, 11]  [1, 1, 0]         False  0.02   
5  clemson_quinnipiac_uncgreen

In [8]:
df.describe()

,bet,mult,payout,profit
count,160.000000,160.000000,160.000000,160.000000
mean,0.016313,9.020867,0.139038,0.010125
std,0.004840,4.823492,0.055591,0.055468
min,0.010000,2.773655,0.027700,-0.110000
25%,0.010000,5.418783,0.095900,-0.032500
50%,0.020000,7.722000,0.165900,0.040000
75%,0.020000,11.264786,0.178500,0.050000
max,0.020000,26.019000,0.260200,0.140000


In [9]:
negative_df = df.loc[df.profit < 0]
negative_df = negative_df.sort_values(by=['event_status', 'profit'], ascending=[False, True])
negative_df.describe()

,bet,mult,payout,profit
count,54.000000,54.000000,54.000000,54.000000
mean,0.015556,4.574093,0.070572,-0.059259
std,0.005016,1.029946,0.031959,0.030148
min,0.010000,2.773655,0.027700,-0.110000
25%,0.010000,3.590734,0.035900,-0.090000
50%,0.020000,4.750325,0.076200,-0.060000
75%,0.020000,5.418783,0.095900,-0.032500
max,0.020000,6.398687,0.136600,-0.010000


In [10]:
positive_df = df.loc[df.profit >= 0]
positive_df = positive_df.sort_values(by=['event_status', 'profit'], ascending=[False, False])
positive_df.describe()

,bet,mult,payout,profit
count,106.000000,106.000000,106.000000,106.000000
mean,0.016698,11.286205,0.173916,0.045472
std,0.004725,4.398412,0.022919,0.021520
min,0.010000,6.195918,0.120500,0.000000
25%,0.010000,7.733488,0.165900,0.040000
50%,0.020000,10.198571,0.174800,0.050000
75%,0.020000,13.314570,0.183200,0.050000
max,0.020000,26.019000,0.260200,0.140000


In [11]:
negative_df.head(200)

,event,index[],result,event_status,bet,mult,payout,profit
6,hawaii_clemson_ncstate_,"[3, 5, 10]","[1, 1, 1]",True,0.01,3.217670,0.0322,-0.09
7,hawaii_clemson_maine_,"[3, 5, 7]","[1, 1, 1]",True,0.02,4.666141,0.0763,-0.04
6,hawaii_clemson_tennessee_,"[3, 5, 8]","[1, 1, 1]",True,0.02,4.666141,0.0763,-0.04
6,hawaii_maine_ncstate_,"[3, 7, 10]","[1, 1, 1]",True,0.02,5.733889,0.1052,-0.03
4,hawaii_tennessee_ncstate_,"[3, 8, 10]","[1, 1, 1]",True,0.02,5.733889,0.1052,-0.03
6,clemson_maine_ncstate_,"[5, 7, 10]","[1, 1, 1]",True,0.02,5.552580,0.0966,-0.03
4,clemson_tennessee_ncstate_,"[5, 8, 10]","[1, 1, 1]",True,0.02,5.552580,0.0966,-0.03
3,ga_south_hawaii_clemson_,"[0, 3, 5]","[1, 1, 1]",True,0.02,5.376623,0.0959,-0.02
2,quinnipiac_fordham_ncstate_,"[6, 9, 10]","[0, 0, 1]",False,0.01,4.137893,0.0414,-0.11
6,bradley_hawaii_ncstate_,"[1, 3, 10]","[0, 1, 1]",False,0.01,3.408347,0.0341,-0.10


In [12]:
positive_df.head(200)

,event,index[],result,event_status,bet,mult,payout,profit
3,ga_south_hawaii_maine_,"[0, 3, 7]","[1, 1, 1]",True,0.02,9.581143,0.1753,0.05
2,ga_south_hawaii_tennessee_,"[0, 3, 8]","[1, 1, 1]",True,0.02,9.581143,0.1753,0.05
3,ga_south_clemson_maine_,"[0, 5, 7]","[1, 1, 1]",True,0.02,9.278182,0.1730,0.05
2,ga_south_clemson_tennessee_,"[0, 5, 8]","[1, 1, 1]",True,0.02,9.278182,0.1730,0.05
2,ga_south_maine_tennessee_,"[0, 7, 8]","[1, 1, 1]",True,0.01,16.533720,0.1857,0.05
2,ga_south_maine_ncstate_,"[0, 7, 10]","[1, 1, 1]",True,0.02,11.401297,0.1895,0.05
0,ga_south_tennessee_ncstate_,"[0, 8, 10]","[1, 1, 1]",True,0.02,11.401297,0.1895,0.05
6,hawaii_maine_tennessee_,"[3, 7, 8]","[1, 1, 1]",True,0.02,8.315063,0.1818,0.05
6,clemson_maine_tennessee_,"[5, 7, 8]","[1, 1, 1]",True,0.02,8.052136,0.1785,0.05
4,maine_tennessee_ncstate_,"[7, 8, 10]","[1, 1, 1]",True,0.02,9.894697,0.2061,0.05


In [13]:
positive_df.groupby('event_status').count()

,event,index[],result,bet,mult,payout,profit
event_status,,,,,,,
False,94,94,94,94,94,94,94
True,12,12,12,12,12,12,12


In [14]:
negative_df.groupby('event_status').count()

,event,index[],result,bet,mult,payout,profit
event_status,,,,,,,
False,46,46,46,46,46,46,46
True,8,8,8,8,8,8,8


In [15]:
#csv

In [16]:
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if type(val) != bool:
        return 'color: black'
    
    color = 'red' if val == True else 'black'
    return 'color: %s' % color

def color_positive_green(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if type(val) != bool:
        return 'color: black'
    
    color = 'green' if val == True else 'black'
    return 'color: %s' % color

In [17]:
neg = negative_df.reset_index(drop=True).style.applymap(color_negative_red)
neg

,event,index[],result,event_status,bet,mult,payout,profit
0,hawaii_clemson_ncstate_,"['3', '5', '10']","[1, 1, 1]",True,0.01,3.21767,0.0322,-0.09
1,hawaii_clemson_maine_,"['3', '5', '7']","[1, 1, 1]",True,0.02,4.66614,0.0763,-0.04
2,hawaii_clemson_tennessee_,"['3', '5', '8']","[1, 1, 1]",True,0.02,4.66614,0.0763,-0.04
3,hawaii_maine_ncstate_,"['3', '7', '10']","[1, 1, 1]",True,0.02,5.73389,0.1052,-0.03
4,hawaii_tennessee_ncstate_,"['3', '8', '10']","[1, 1, 1]",True,0.02,5.73389,0.1052,-0.03
5,clemson_maine_ncstate_,"['5', '7', '10']","[1, 1, 1]",True,0.02,5.55258,0.0966,-0.03
6,clemson_tennessee_ncstate_,"['5', '8', '10']","[1, 1, 1]",True,0.02,5.55258,0.0966,-0.03
7,ga_south_hawaii_clemson_,"['0', '3', '5']","[1, 1, 1]",True,0.02,5.37662,0.0959,-0.02
8,quinnipiac_fordham_ncstate_,"['6', '9', '10']","[0, 0, 1]",False,0.01,4.13789,0.0414,-0.11
9,bradley_hawaii_ncstate_,"['1', '3', '10']","[0, 1, 1]",False,0.01,3.40835,0.0341,-0.1


In [18]:
pos = positive_df.reset_index(drop=True).style.applymap(color_positive_green)
pos

,event,index[],result,event_status,bet,mult,payout,profit
0,ga_south_hawaii_maine_,"['0', '3', '7']","[1, 1, 1]",True,0.02,9.58114,0.1753,0.05
1,ga_south_hawaii_tennessee_,"['0', '3', '8']","[1, 1, 1]",True,0.02,9.58114,0.1753,0.05
2,ga_south_clemson_maine_,"['0', '5', '7']","[1, 1, 1]",True,0.02,9.27818,0.173,0.05
3,ga_south_clemson_tennessee_,"['0', '5', '8']","[1, 1, 1]",True,0.02,9.27818,0.173,0.05
4,ga_south_maine_tennessee_,"['0', '7', '8']","[1, 1, 1]",True,0.01,16.5337,0.1857,0.05
5,ga_south_maine_ncstate_,"['0', '7', '10']","[1, 1, 1]",True,0.02,11.4013,0.1895,0.05
6,ga_south_tennessee_ncstate_,"['0', '8', '10']","[1, 1, 1]",True,0.02,11.4013,0.1895,0.05
7,hawaii_maine_tennessee_,"['3', '7', '8']","[1, 1, 1]",True,0.02,8.31506,0.1818,0.05
8,clemson_maine_tennessee_,"['5', '7', '8']","[1, 1, 1]",True,0.02,8.05214,0.1785,0.05
9,maine_tennessee_ncstate_,"['7', '8', '10']","[1, 1, 1]",True,0.02,9.8947,0.2061,0.05
